# 4章 word2vecの高速化

本章では，word2vecの高速化に主眼を置き，word2vecの改善に取り組む．

#### 2つの改良を加えて，本物のword2vecを作成する
- Embeddingレイヤという新しいレイヤを導入する
- Negative Samplingという新しい損失関数を導入する


最終的には，PTBデータセット（実用的なサイズのコーパス）を対象に学習を行う．

→単語の分散表現の良さを実際に評価する．

## 4.1 word2vecの改良①

### 4.1.1 Embeddintレイヤ

仮に100万語の語彙数からなるコーパスがあれば，単語のone-hot表現の次元数は100万になる．そして巨大なベクトルと重み行列の積を計算する必要があった．しかし，ここで行っているのことは，行列の特定の行を抜き出すことだけ，なのでone-hot表現への変換と，MatMulレイヤでの行列の乗算は必要なさそう．

### 4.1.2 Embeddingレイヤの実装

In [3]:
# 重みから特定の行を抜き出すには．．．
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W)
print(W[2])
print(W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]


In [5]:
# 複数の行を抜き出す  ミニバッチを想定した実装
idx = np.array([1, 0, 3, 0])
print(W[idx])

[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [10]:
import numpy as np
a = np.arange(21)
print(a)
a = a.reshape(7, 3)
print(a)

# embedding層の実装
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        print(dW)
        dW[...] = 0
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        # もしくは
        # np.add.at(dW, self.idx, dout)
        
        return None
a = Embedding(a)

a.forward(np.array([0, 1, 2]))

print(a.params)
print(a.grads)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])]
[array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])]


Embeddingレイヤの順伝播は，重みWの特定の行を抜き出すだけ，つまり重みの特定の行のニューロンだけを何の手も加えず次の層へと流したことになる．逆伝播の場合は，前の層から伝わってきた勾配を次の層へそのまま流すだけ．しかし，前層から伝わる勾配を，重みの勾配dWの特定の行（idx）に設定するようにする．

まとめると，入力側のMatMulレイヤをEmbeddingレイヤに切り替えて無駄な計算を省いたという話．

## 4.2 word2vecの改良② 

残るボトルネックは，中間層以降の処理，行列の積とSoftmaxレイヤの計算である．
#### Negative Sampling(負例サンプリング)
Softmaxの代わりにNegative Samplingを用いることで，語彙数がどれだけ多くなったとしても，計算量を少なく一定に抑えることができる．

### 4.2.1 中間層以降の計算の問題点

前節に引き続き，語彙数が100万，中間層のニューロン数が100のときのword2vec(CBOWモデル)を考える．

入力層と出力層には100万個のニューロンが存在する．入力層の計算についてはEmbeddingレイヤを導入することで，無駄を省く方法を見てきた．残る問題としては中間層以降の処理だが．中間層に関しては以下の2つの場所で多くの計算時間が必要になる．

- 中間層のニューロンと重み行列($W_out$)の積
- Softmaxレイヤの計算

巨大な行列の積は，多くの計算時間と多くのメモリを必要とする．この処理を軽くする必要がある．

#### Softmaxの計算量の大きさを式で確認する
### $$
y_k = \frac{exp(s_k)}{\sum_{i=0}^{1000000}exp(s_i) \quad}
$$

### 4.2.2 多値分類から二値分類へ

#### Negative samplingを理解する上で重要なポイント
「多値分類」を「二値分類」で近似すること．

ターゲットとなる単語だけのスコアを求めるニューラルネットワークを構築し，ターゲットの単語が答えであるか，否かという二値分類問題を近似的に作り出す．

上記を考慮すると中間層と出力側の重みの行列の積は，ターゲットに対応する列（単語ベクトル）だけを抽出し，その抽出したベクトルと中間層のニューロンとの内積を計算すればよいことになる．そしてこれが最終的なスコアになる．

### 4.2.3 シグモイド関数と交差エントロピー誤差

#### 多値分類
出力層には「ソフトマックス関数」，損失関数には「交差エントロピー誤差」を用いる．
#### 二値分類
出力層には「シグモイド関数」，損失関数には「交差エントロピー誤差」を用いる．

#### シグモイド関数
$$ 
y = \frac{1}{1+exp(-x)}
$$

#### 交差エントロピー誤差
$$
L = -(tlogy+(1-t)log(1-y))
$$

#### 逆伝播時に伝播するy - t
- y: ニューラルネットワークが出力した確率
- t: 正解ラベル

つまりy - tはその2つの誤差．誤差が前レイヤに流れたとき，大きい誤差の場合は大きく学習し，小さい誤差の場合は小さく学習するようになる．

#### y - tというきれいな結果
- シグモイド関数と交差エントロピー誤差
- ソフトマックス関数と交差エントロピー誤差
- 恒等関数と2乗和誤差

上の3つの組み合わせはどれも逆伝播時にはy - tの値が伝播する．

### 4.2.4 多値分類から二値分類へ（実装編）

#### Embedding Dotレイヤ
Embeddingレイヤとdot演算(内積)の2つの処理を合わせたレイヤ．中間層以降の処理だけにフォーカスして描画する．EmbeddingDotレイヤを使って，Embeddingレイヤと内積の計算をまとめて行う．

In [11]:
# EmbiddingDotレイヤ
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 4.2.5 Negative Sampling

本当に行いたいことは，正例についてはSigmoidレイヤの出力を1に近づけ，負例については，Sigmoidレイヤの出力を0に近づけること．

コンテキストに対して誤ったターゲットである場合の確率は低い値であることが望まれる．

#### 多値分類の問題を二値分類として扱うためには，「正しい答え（正例）」と「間違った答え（負例）」のそれぞれに対して，正しく分類できる必要がある．そのため，正例と負例の両者を対象として問題を考える必要がある．

#### Negative Sampling
すべての負例を対象にして，二値分類の学習を行うことはしない．その代わりに近似解としていくつかピックアップする．つまりネガティブな例を少数サンプリングする．損失関数は，正例をターゲットとした場合の損失といくつかサンプリングした負例の損失を足し合わせたものになる．

### 4.2.6 Negative Samplingのサンプリング方法

#### 負例のサンプリング
ランダムにサンプリングするよりも良い方法が知られている．それはコーパスの統計データに基づいたサンプリングを行うこと．コーパス中でよく使われる単語は抽出されやすくし，コーパスの中であまり使われない単語は抽出されにくくする．

#### word2vecで提案されたNegative samplingにおける確率分布の式
### $$
P^{'}(W_i) = \frac{P(w_i^{0.75})}{\sum_{j}^{n}P(w_j)^{0.75}}
$$

#### UnigramSamplerクラス
実装内容に興味がある場合には各自参照．

In [13]:
# UnigramSamplerクラスを使って処理してみる

from negative_sampling_layer import UnigramSampler

# 使用する引数は3つ
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[0 3]
 [4 0]
 [1 3]]


### 4.2.7 Negative Samplingの実装

最後にNegativeSamplingの実装を行う．

In [16]:
# Negative Sampling

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        # 負例samplesize+正例1つ分のレイヤをそれぞれリストで保持する
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size \
                                                            +1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size \
                                                               + 1)]
        self.params, self.grads = [], []
        # このレイヤで使用する配列をそれぞれ配列にまとめる．
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
        
        print(self.params, self.grads)
        
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 正例のフォワード
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 負例のフォワード
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    # 逆伝播
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backword(dout)
            dh += l1.backward(dscore)
            
        return dh

## 4.3 改良版word2vecの学習

- Embeddingレイヤ
- Negative Sampling

2つの手法を取り入れて，ニューラルネットワークを作成する．そしてPTBデータセットを使って学習し，より実用的な単語の分散表現を獲得する．

### 4.3.1 CBOWモデルの実装

In [24]:
# CBOWクラス

import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, \
                                           sample_size=5)
        
        # すべての重みと勾配を配列にまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in
        
    # 順伝播
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    # 逆伝播
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### 4.3.2 CBOWモデルの学習コード

最後に，CBOWモデルの学習を行う．

In [ ]:
# # ニューラルネットワークの学習を行うだけ

# import sys
# sys.path.append('..')
# import numpy as np
# from common import config
# import pickle
# from common.trainer import Trainer
# from common.optimizer import Adam
# from cbow import CBOW
# from common.util import create_contexts_target, to_cpu, to_gpu
# from dataset import ptb

# # ハイパーパラメータの設定
# window_size = 5
# hidden_size = 100
# batch_size = 100
# max_epoch = 10

# # データの読み込み
# corpus, word_to_id, id_to_word = ptb.load_data('train')
# vocab_size = len(word_to_id)

# contexts, target = create_contexts_target(corpus, window_size)

# # モデルなどの生成
# model = CBOW(vocab_size, hidden_size, window_size, corpus)
# optimizer = Adam()
# trainer = Trainer(model, optimizer)

# # 学習開始
# trainer.fit(contexts, target, max_epoch, batch_size)
# trainer.plot()

# # 後ほど利用できるように，必要なデータを保存
# word_vecs = model.word_vecs

# params = {}
# params['word_vecs'] = word_vecs.astype(np.float16)
# params['word_to_id'] = word_to_id
# params['id_to_word'] = id_to_word
# pkl_file = 'cbow_params.pkl'
# with open(self_pkl_file, 'wb') as f:
#     pickle.dump(params, f, -1)

### 4.3.3 CBOWモデルの評価

前節で学習した単語の分散表現を評価してみる．

In [ ]:
# most_similar()メソッドを使用していくつかの単語に対して最も距離の近い単語

import sys
sys.path.append('..')
from commmon.util import most_similar
import pickle

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']
    
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

## 4.4 word2vecに関する残りのテーマ

ここまでで，word2vecの仕組みや実装についての説明は終わり．

### 4.4.1 word2vecを使ったアプリケーションの例

#### 転移学習
単語の分散表現を他の分野のタスクに適用させる．テキスト分類や文書クラスタリング，品詞タグ付け，感情分析などの自然言語タスクにおいて，単語をベクトルに変換する最初のステップでは，学習済みの単語の分散表現を利用できる．単語の分散表現はそれらのタスクにおいて素晴らしい結果をもたらす．単語の分散表現については最初にwikiやgooglenewsなどの大きなコーパスで学習を行っておく．

さらに単語の分散表現の利点として，単語を固定長のベクトルに変換できることにある．これは文章に対しても行える．自然言語をベクトルに変換できれば，一般的な機械学習の手法が適用できる．

### 4.4.2 単語ベクトルの評価方法

#### 単語の分散表現の良さをどのように評価するか？？
単語の分散表現の良さを評価するにあたり，現実的なアプリケーションとは切り離して評価を行う．よく用いられる評価手法は，「類似性」や「類推問題」．
#### 単語の類似性の評価
人間が作成した単語類似度の評価セットを使って評価する．
#### 類推問題による単語ベクトルの評価結果からわかること
- モデルによって精度が異なる（コーパスに応じて最適なモデルを選ぶ）
- コーパスが大きいほど良い結果になる（ビックデータは常に望まれる）
- 単語ベクトルの次元数は適度な大きさが必要（大きすぎても精度が悪くなる）

## 4.5 まとめ

#### 本章で学んだこと
- Embeddingレイヤは単語の分散表現を格納し，順伝播において該当する単語IDベクトルを抽出する
- word2vecでは語彙数の増加に比例して計算量が増加するので，近似計算を行う高速な手法を使うといい
- NagativeSamplingは負例をいくつかサンプリングする手法であり，これを利用すれば多値分類を二値分類として扱うことができる
- word2vecによって得られた単語の分散表現は，単語の意味が埋め込まれたものであり，似たコンテキストで使われる単語は単語ベクトルの空間上で近い場所に位置するようになる
- word2vecの単語の分散表現は，類推問題をベクトルの加算と減算によって解ける性質を持つ
- word2vecは転移学習の点で特に重要であり，その単語の分散表現はさまざまな自然言語処理のタスクに利用できる